In [0]:
from pyspark.sql.types import StringType
import pyspark.sql.functions as F
import requests
import json
import urllib.parse

#### Database Schema

<img src="https://i.ibb.co/hRqLR8d/Schema.png" alt="Schema" border="0">

In [0]:
raw_df = spark.read.format("delta").load("/user/hive/warehouse/scientific_publications")
display(raw_df.limit(5))

_id,abstract,authors,doi,fos,isbn,issn,issue,keywords,lang,n_citation,page_end,page_start,pdf,references,title,url,venue,volume,year
53e99784b7602d9701f3e3f5,null,null,null,null,null,null,null,List(),en,0,null,null,null,null,3GIO.,null,"List(null, null, null, null, null, null, null, null, null, null, null, null, 0)",null,2011
53e99784b7602d9701f3e133,"Drought is the first place in all the natural disasters in the world. It is especially serious in North China Plain. In this paper, different soil water content control levels at winter wheat growth stages are performed on Gucheng Ecological-Meteorological Integrated Observation Experiment Station of CAMS, China. Some canopy parameters, including growth conditions, dry weight, physiological parameters and hyperspectral reflectance, are measured from erecting stage to milk stage for winter wheat in 2009. The relationship between canopy parameters and soil relative moisture, canopy water content and water indices of winter wheat are established. The results show that some parameters, such as SPAD and dry weight of leaves, decrease with the increasing of soil relative moisture, while other parameters, including dry weight of caudexes, above ground dry weight, height, photosynthesis rate, intercellular CO 2 concentration, stomatal conductance and transpiration rate, increase corresponding to the soil relative moisture. Obvious linear relationship between stomatal conductance and transpiration rate is established with 45 samples, which R2 reaches to 0.6152. Finally, the fitting equations between canopy water content and water indices are regressed with b5, b6 and b7 of MODIS bands. The equations are best with b7 and worst with b5. So the fitting equations with b7 can be used to inverse the canopy water content of winter wheat using MODIS or other remote sensing images with similar bands range to MODIS in Hebei Province. © 2011 IEEE.","List(List(53f45728dabfaec09f209538, null, null, null, Peijuan Wang, null, null, null, null, null, null, null, null, null, null), List(5601754345cedb3395e59457, null, null, null, Jiahua Zhang, null, null, null, null, null, null, null, null, null, null), List(53f38438dabfae4b34a08928, null, null, null, Donghui Xie, null, null, null, null, null, null, null, null, null, null), List(5601754345cedb3395e5945a, null, null, null, Yanyan Xu, null, null, null, null, null, null, null, null, null, null), List(53f43d25dabfaeecd6995149, null, null, null, Yun Xu, null, null, null, null, null, null, null, null, null, null))",10.1109/IGARSS.2011.6049503,"List(Agronomy, Moisture, Hydrology, Environmental science, Dry weight, Water content, Stomatal conductance, Transpiration, Irrigation, Soil water, Canopy)",,,null,"List(canopy parameters, canopy spectrum, different soil water content control, winter wheat, irrigation, hydrology, radiometry, moisture, indexes, vegetation, indexation, dry weight, soil moisture, water content, indexing terms, spectrum, natural disaster)",en,0,1933,1930,null,null,The relationship between canopy parameters and spectrum of winter wheat under different irrigations in Hebei Province.,List(http://dx.doi.org/10.1109/IGARSS.2011.6049503),"List(53a7297d20f7420be8bd4ae7, null, null, International Geoscience and Remote Sensing Symposium, null, null, null, IGARSS, null, null, null, null, 0)",null,2011
53e99784b7602d9701f3e151,,"List(List(53f46797dabfaeb22f542630, null, null, null, Jairo Rocha, null, null, null, null, null, null, null, null, null, null), List(54328883dabfaeb4c6a8a699, null, null, null, Theo Pavlidis, null, null, null, null, null, null, null, null, null, null))",10.1109/ICDAR.1993.395663,"List(Intelligent character recognition, Pattern recognition, Computer science, Feature (computer vision), Document processing, Handwriting recognition, Optical character recognition, Feature extraction, Feature (machine learning), Artificial intelligence, Intelligent word recognition)",,,,"List(handwriting recognition, prototypes, image segmentation, com

In [0]:
#Date, Keyword, Type, Publication, Venue, FOS, Organization, Author, Langue, AuthorRank
raw_df.columns

Out[107]: ['_id',
 'abstract',
 'authors',
 'doi',
 'fos',
 'isbn',
 'issn',
 'issue',
 'keywords',
 'lang',
 'n_citation',
 'page_end',
 'page_start',
 'pdf',
 'references',
 'title',
 'url',
 'venue',
 'volume',
 'year']

In [0]:
#Filter out titles that have less than 2 words
filtered_df = raw_df.withColumn("title_word_count", F.size(F.split("title", " "))).filter("title_word_count > 1")
display(filtered_df.limit(5))

_id,abstract,authors,doi,fos,isbn,issn,issue,keywords,lang,n_citation,page_end,page_start,pdf,references,title,url,venue,volume,year,title_word_count
53e99967b7602d97021a560a,"Ab initio predictions of secondary structures in proteins have to combine local predictions, based on short fragments of the protein sequence, with consistency restrictions, as not all locally plausible predictions may be simultaneously true. We use the fact that secondary structures are patterns of hydrogen bonds and that a single residue can participate in hydrogen bonds of at most one secondary structure. Consistency of fixed-sized pieces of secondary structures is the easiest to approximate and we formalize it as 1-2 matching problem. Consistency of entire secondary structures is a version of set packing. We also investigate how to form a simple problem if we add the requirement that the secondary structure and the loops that connect them fit together in a metric space. Every problem that we investigated is MAX-SNP hard and it has a constant factor approximation. Computational experience suggests that in biological instances, we can find nearly optimal solutions using heuristics.","List(List(53f45d3adabfaee0d9c0cb90, null, berman@cse.psu.edu, 5b86b6c2e1cd8e14a34ca959, Piotr Berman, null, null, null, null, The Pennsylvania State University, University Park, PA, USA, null, 5f71b2d91c455f439fe3eb4f, List(Penn State Univ, University Pk, PA 16802 USA), null, null), List(53f436cbdabfaedf43582e9f, null, null, 5b86b6c2e1cd8e14a34ca959, Jieun Jeong, null, null, null, null, The Pennsylvania State University, University Park, PA, USA, null, 5f71b2d91c455f439fe3eb4f, List(Penn State Univ, University Pk, PA 16802 USA), null, null))",10.1007/s00453-007-9068-8,"List(Algorithm, Heuristics, Set packing, Metric space, Ab initio, Hydrogen bond, Protein secondary structure, Mathematics, Protein structure, Theory of computation)",,0178-4617,1,"List(Secondary Structure, Maximum Weight, Local Prediction, Horizontal Edge, Connector Edge)",en,2,34,16,null,"List(53e9bdcdb7602d9704a7e6ae, 53e99d5eb7602d970261a1c4, 53e9bdcdb7602d9704a7e6ae, 53e99e3eb7602d970270422e, 53e9adffb7602d970380a762, 53e9b593b7602d97040d9071, 53e99c6fb7602d9702522122, 55a43e58612ca648688dc943, 53e99822b7602d97020431be)",Consistent Sets of Secondary Structures in Proteins,"List(http://dx.doi.org/10.1007/s00453-007-9068-8, http://dx.doi.org/https://doi.org/10.1007/s00453-007-9068-8, https://link.springer.com/article/10.1007/s00453-007-9068-8, https://dl.acm.org/citation.cfm?id=3118965&picked=prox&preflayout=flat, http://www.webofknowledge.com/)","List(539078f220f770854f5a882f, null, null, null, null, null, null, Algorithmica, null, null, null, null, 0)",53,2009,7
53e99960b7602d97021a53fa,"In this work a new method of feature extraction for an interactive and adaptive recognizer for on-line handwritten alphanumeric characters has been proposed. The system is suitable for use in conjunction with magnetic pen based devices for inputting data to a data processing system or a computer terminal. The features are extracted from dynamically changing locations of the writing device. The new method of feature extraction is simple, computationally light and fast enough for adaptive on-line use. Extracted features are robust with respect to all possible distortions like shape, size, and orientation. For simulation experiment, numerals 0-9 are used. A single hidden layer feed forward neural network trained by Quickprop algorithm, a variation of error back propagation is used for recognition. Very high recognition rates, even for highly distorted samples have been achieved confirming high generalization capability of the extracted feature set.","List(List(53f36626dabfae4b3499aa7a, null, null, 5b86b9c0e1cd8e14a3625bbe, Basabi Chakraborty, null, null, null, null, Faculty of Software and Information Science, Iwate Prefectural University, 152-52 Aza Sugo, Takizawamura, Iwate 020-0193, Japan, null, 5f71b4211c455f439fe47d1e,

In [0]:
def get_author_from_dblp(title, rank):
    URL = "http://dblp.org/search/publ/api?q=" + title.replace(" ", "+") + "&format=json"
    print(URL)
    r = requests.get(url = URL)
    data = r.json()
    #if 'hit' in data['result']['hits'] and len(data['result']['hits']['hit']) > 0:
        #if len(data['result']['hits']['hit'][0]['info']['authors']['author']) > 0:
    try:
        return data['result']['hits']['hit'][0]['info']['authors']['author'][rank-1]['text']
    except:
        return None
    
get_author_from_dblpUDF = F.udf(lambda x, y:get_author_from_dblp(x, y), StringType()) 

In [0]:
#Author transformations
authors = (filtered_df.limit(100).select("authors", "title", F.posexplode(F.col("authors")).alias("rank", "authors_exp"))
           .withColumn("rank", F.col("rank") + 1)
           .select("authors_exp.*","*")
           .select("_id", "rank", "name", "title")
           .withColumn("name", F.initcap(F.col("name")))
           .withColumn("dblp_name", F.when(F.col("name").rlike(r"^\p{L}\.?(-\p{L}\.?)?\s.+"), 
                                            get_author_from_dblpUDF(F.col("title"), F.col("rank")))
                          .otherwise(F.col("name")))
           .withColumn("last_name_raw", F.regexp_extract(F.col("name"), r"^.+\s(\S+)(\sJr\.)?$", 1))
           .withColumn("dblp_name", F.when(F.col("dblp_name") == "null", F.col("name")).otherwise(F.col("dblp_name")))
           .withColumn("dblp_name", F.regexp_extract(F.col("dblp_name"), r"^(\D+)(\s\d\d\d\d)?$", 1))
           .withColumn("dblp_last_name", F.regexp_extract(F.col("dblp_name"), r"^.+\s(\S+)(\sJr\.)?$", 1))
           .withColumn("name", F.when(F.col("last_name_raw") == F.col("dblp_last_name"), 
                                      F.col("dblp_name")).otherwise(F.col("name")))
           .withColumn("first_name", F.regexp_extract(F.col("name"), r"^(\S+)\s.+$", 1))
           .withColumn("last_name", F.regexp_extract(F.col("name"), r"^.+\s(\S+)$", 1))
           .withColumn("middle_name", F.regexp_extract(F.col("name"), r"^\S+\s(\S+)\s\S+$", 1))
           .select("_id", "first_name", "last_name", "middle_name")
          )
          
display(authors)

_id,first_name,last_name,middle_name
53f45d3adabfaee0d9c0cb90,Piotr,Berman,
53f436cbdabfaedf43582e9f,Jieun,Jeong,
53f36626dabfae4b3499aa7a,Basabi,Chakraborty,
53f428c6dabfaeb22f3d18be,Goutam,Chakraborty,
53f4489fdabfaeee22a007dc,T.,Merriam,
null,Julius,Müller,
53f42abbdabfaeb1a7b680dc,Thomas,Magedanz,
53f4bb8cdabfaedce565271a,Jens,Fiedler,
54408649dabfae7f9b33f2b4,Ming,Chen,
null,Guangwen,Yang,


In [0]:
res = get_author_from_dblp("Global Approach for Compiled Bit-True Simulation of DSP Systems", 4)
print(res)

http://dblp.org/search/publ/api?q=Global+Approach+for+Compiled+Bit-True+Simulation+of+DSP+Systems&format=json
J. A. Peperstraete


In [0]:
def get_organization(name):
    try:
        URL = "https://api.ror.org/organizations?query=" + name.replace(" ", "+")
        r = requests.get(url = URL)
        data = r.json()
        if data['number_of_results'] == 0:
            return "No results"
        return data['items'][0]["name"]+";"+data['items'][0]["addresses"][0]["city"]+";"+data['items'][0]["country"]["country_name"]
    except:
        return "No results"
get_organizationUDF = F.udf(lambda x:get_organization(x), StringType()) 

In [0]:
#Organization transformations
organization = (filtered_df.limit(1000).select("authors", "title", F.explode(F.col("authors")).alias("authors_exp"))
               .select("authors_exp.*","*")
               .select("org")
               .filter(F.col("org").isNotNull())
               .dropDuplicates()
                # ., +, *, ?, ^, $, (, ), [, ], {, }, |, \
               .withColumn("strip_org", F.regexp_replace(F.col("org"), r'[\+-=#&\|><!\(\)\{\}\[\]\^"~\*\?:\\/]', " "))
               .withColumn("api_org", get_organizationUDF(F.col("strip_org")))
               .withColumn("name", F.when(F.col("api_org") == "No results", F.col("org"))
                           .otherwise(F.regexp_extract(F.col("api_org"), r"^(.+);.+;.+$", 1)))
               .withColumn("city", F.when(F.col("api_org") == "No results", "")
                           .otherwise(F.regexp_extract(F.col("api_org"), r"^.+;(.+);.+$", 1)))
               .withColumn("country", F.when(F.col("api_org") == "No results", "")
                           .otherwise(F.regexp_extract(F.col("api_org"), r"^.+;.+;(.+)$", 1)))
               .select("name", "city", "country")
               )

display(organization)

name,city,country
Pennsylvania State University,State College,United States
Iwate Prefectural University,Takizawa,Japan
Fraunhofer Institute for Open Communication Systems,Berlin,Germany
Technical University of Berlin,Berlin,Germany
University of Computer Studies Yangon,Yangon,Myanmar
New Europe College,Bucuresti,Romania
University of Houston,Houston,United States
Florida Atlantic University,Boca Raton,United States
University of Tsukuba,Tsukuba,Japan
Tama Art University,Tokyo,Japan


In [0]:
def get_organization2(name):
    URL = "https://api.ror.org/organizations?query=" + urllib.parse.quote(name)
    r = requests.get(url = URL)
    data = r.json()
    if data['number_of_results'] == 0:
        return "No results"
    return data['items'][0]["name"]+";"+data['items'][0]["addresses"][0]["city"]+";"+data['items'][0]["country"]["country_name"]

res = get_organization2("SEM, Tsinghua University, Beijing, China")

print(res)

Tsinghua Holdings (China);Beijing;China
